1. Setup

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader

import os
from PIL import Image
import pandas as pd


# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


2. Data Loading

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Pre-process the data

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [10]:

import os
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

data_path = '/content/drive/MyDrive/Study/ML_TeamProject/newData_224_splited'

# Define batch size
batch_size = 32

# Create custom Dataset objects for your data
train_dataset = ImageFolder(
    os.path.join(data_path, 'train'),
    transform=transform
)

val_dataset = ImageFolder(
    os.path.join(data_path, 'val'),
    transform=transform
)

test_dataset = ImageFolder(
    os.path.join(data_path, 'test'),
    transform=transform
)

# Create DataLoader objects for batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)




############
sample_batch = next(iter(train_loader))  # Get a batch of data
images, labels = sample_batch  # Unpack the batch into images and labels

# Check the size of the first image
first_image = images[0]  # Assuming you want to check the first image in the batch
image_size = first_image.shape  # Get the size of the image

print(f"Image size: {image_size}")


# Image size: torch.Size([3, 224, 224])
# len(train_loader) : 78
# len(val_loader) : 7
# len(test_loader) : 14



Image size: torch.Size([3, 32, 32])


3. Define the CNN

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * 28 * 28, 512)  # Update the linear layer's input size
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 256 * 28 * 28)  # Update the size to match the feature map size
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

num_classes = 215  # Number of classes in your dataset
net = Net(num_classes=num_classes).to(device)


4. Define the Loss function and Optimizer

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
# Reduce LR every 10 epochs by 5%, Dynamically setting lr


5. Train the network

In [14]:

import torch.optim as optim

num_epochs = 20

for epoch in range(num_epochs):
    net.train()

    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)  #input = image, label = class_name

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        if i % 10 == 9:
            interval_accuracy = 100 * correct_train / (10 * train_loader.batch_size)
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 10:.5f}, accuracy: {interval_accuracy:.2f}%")
            running_loss = 0.0
            correct_train = 0

    train_accuracy = 100 * correct_train / total_train
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss / len(train_loader)}, Training accuracy: {train_accuracy:.2f}%")

    # Validation loop
    net.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for data in val_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    average_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Validation loss: {average_val_loss:.3f}, Validation accuracy: {val_accuracy:.2f}%")

print('Finished Training')


#  결과 정리
# lr = 0.001, epoch=10
# [1, 10] loss: 5.38286, accuracy: 0.94%
# [1, 20] loss: 5.37374, accuracy: 0.62%
# [1, 30] loss: 5.37406, accuracy: 0.62%
# [1, 40] loss: 5.37909, accuracy: 0.94%
# [1, 50] loss: 5.37122, accuracy: 0.62%

# lr=0.005, epoch=20
# [1, 10] loss: 5.37653, accuracy: 0.00%
# [1, 20] loss: 5.37914, accuracy: 0.00%
# [1, 30] loss: 5.38180, accuracy: 0.00%
# [1, 40] loss: 5.38372, accuracy: 0.31%
# [1, 50] loss: 5.39082, accuracy: 0.00%
# [1, 60] loss: 5.38192, accuracy: 0.31%
# [1, 70] loss: 5.38536, accuracy: 0.00%
# Epoch 1/20, Training Loss: 0.5522051896804419, Training accuracy: 0.00%
# Epoch 1/20, Validation loss: 5.372, Validation accuracy: 0.48%


RuntimeError: ignored

In [ ]:
torch.save(net, 'model2.pth')

In [ ]:
# Assuming the model's class is `Net` as in previous examples
model = torch.load('model2.pth')
model.eval()  # Set the model to evaluation mode


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

6. Test the network on test data

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 58 %
